<center><h1>1-ab: SIFT & BoW</h1></center>

<center><h2><a href="https://rdfia.github.io/">Course link</a></h2></center>

To keep your modifications in case you want to come back later to this colab, do *File -> Save a copy in Drive*.



In [ ]:
!wget http://webia.lip6.fr/~dancette/deep-learning/assets/TP1-2-data.zip
!unzip TP1-2-data.zip

!wget https://raw.githubusercontent.com/rdfia/rdfia.github.io/master/code/1-ab/tools.py

--2022-01-01 01:21:30--  http://webia.lip6.fr/~dancette/deep-learning/assets/TP1-2-data.zip
Resolving webia.lip6.fr (webia.lip6.fr)... 132.227.201.33
Connecting to webia.lip6.fr (webia.lip6.fr)|132.227.201.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86446976 (82M) [application/zip]
Saving to: ‘TP1-2-data.zip.1’

TP1-2-data.zip.1    100%[===================>]  82.44M  25.5MB/s    in 3.2s    

2022-01-01 01:21:34 (25.5 MB/s) - ‘TP1-2-data.zip.1’ saved [86446976/86446976]

Archive:  TP1-2-data.zip
replace data/Scene/PARoffice/image_0091.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np
%run 'tools.py'
from os.path import join

data_path = "data"

import numpy as np
import scipy.stats as st
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance_matrix
from sklearn.preprocessing import normalize


# Part 1 : SIFT

In [ ]:
# example images
I = read_grayscale(join(data_path, 'tools.tiff'))
I2 = read_grayscale(join(data_path, 'Scene/CALsuburb/image_0205.jpg'))
plt.imshow(I)

In [ ]:
print(len(I))
print(len(I[0]))
from math import *

In [ ]:
Mx = 1/4*np.array([[-1, 0 ,1] ,[-2 ,0 ,2] , [-1, 0, 1]])
My = 1/4*np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
hx = np.array([1, 2, 1])
hy = np.array([-1, 0, 1])

In [ ]:
def compute_grad(I):
    Ix = conv_separable(I, hy, hx)
    Iy = conv_separable(I, hx, hy)
    return Ix, Iy

# Example
Ix, Iy = compute_grad(I)
plt.imshow(Ix)
plt.colorbar()
plt.show()
plt.imshow(Iy)
plt.colorbar()
plt.show()

In [ ]:
def compute_grad_mod_ori(I):
    Ix, Iy = compute_grad(I)
    Gn = np.sqrt(np.square(Ix) + np.square(Iy))
    Go = compute_grad_ori(Ix, Iy, Gn)
    return Gn, Go

In [ ]:
Gn, Go = compute_grad_mod_ori(I)
plt.imshow(Gn)
plt.show()
plt.imshow(Go)
plt.show()

In [ ]:
def compute_histogram(g_n, g_o):
    hist = np.zeros((8))
    n = len(g_o)
    for i in range(len(g_o)):
      for j in range(len(g_o[0])):
        goij = g_o[i,j]
        if goij != -1:
          hist[goij] += g_n[i,j]
    return hist

In [ ]:
def gkern(kernlen, nsig):
    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()
  
mask = 237*gkern(16, 0.5)

In [ ]:
def compute_sift_region(Gn, Go, mask=mask):
    t_min=.5
    t_max=.2
    with_l2 = True

    patch_size = 16
    sift = np.zeros((128)) 

    if mask is not None:
        Gn = Gn * mask
    
    idx = 0
    for k in range(0, patch_size, 4):
        for l in range(0, patch_size, 4):
            hist = compute_histogram(Gn[l:l+4,k:k+4], Go[l:l+4,k:k+4])            
            sift[idx:idx+8] = hist
            idx += 8

    norm = np.linalg.norm(sift, ord=2)
    # min thresholding on norm
    if norm <= t_min:
        return np.zeros((128))
    # l2-normalization
    if with_l2:
        sift = sift / norm
    # max thresholding on values
    sift[sift >= t_max] = t_max
    # l2-normalization
    if with_l2:
        norm = np.linalg.norm(sift, ord=2)
        sift = sift / norm
    return sift

In [ ]:
# Example of viz of SIFTs
# set gausm to True to apply mask weighting of gradients
#display_sift_region(I,           compute_grad_mod_ori, compute_sift_region, x=200, y=78, gausm=False)
#display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=100, y=124, gausm=False)
#display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=125, y=100, gausm=False)
#display_sift_region(marche_im(), compute_grad_mod_ori, compute_sift_region, x=121, y=121, gausm=False)
display_sift_region(toy_im(),    compute_grad_mod_ori, compute_sift_region, x=95, y=95, gausm=False)

In [ ]:
def compute_sift_image(I):
    x, y = dense_sampling(I)
    im = auto_padding(I)
    m = gaussian_mask()
    
    # Here, compute on the global image (norm, gradients)
    Gn, Go = compute_grad_mod_ori(I)
    sifts = np.zeros((len(x), len(y), 128))
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            GnR = Gn[xi:xi+16, yj:yj+16]
            GoR = Go[xi:xi+16, yj:yj+16]
            sifts[i, j, :] = compute_sift_region(GnR, GoR, mask=None)
    return sifts

# Compute SIFT

The computation can take several minutes.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
max_images_per_class = 10  # You can change this to None to compute the whole dataset (takes quite some time)

save_path = "/content/gdrive/My Drive/rdfia/"
dir_sc = os.path.join(data_path, 'Scene')
dir_sift = os.path.join(save_path, 'sift')
inames, ilabls, cnames = load_dataset(dir_sc, images_per_class=max_images_per_class)
sifts_list_by_image = compute_load_sift_dataset(dir_sc, dir_sift, inames, compute_sift_image)

# Part 2 : Visual dictionnary

In [ ]:
from sklearn.cluster import KMeans

path_vdict = os.path.join(save_path, 'kmeans', 'vdict.npy')
os.makedirs(os.path.join(save_path, 'kmeans'), exist_ok=True)

In [ ]:
# Code here the `compute_visual_dict` function
def compute_visual_dict(sift, n_clusters=1000, n_init=1, verbose=1):
    # reorder data
    dim_sift = sift[0].shape[-1]
    sift = [s.reshape(-1, dim_sift) for s in sift]
    sift = np.concatenate(sift, axis=0)
    # remove zero vectors
    keep = ~np.all(sift==0, axis=1)
    sift = sift[keep]
    # randomly pick sift
    ids, _ = compute_split(sift.shape[0], pc=0.05)
    sift = sift[ids]

    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, verbose=verbose).fit(sift)
    vdict = kmeans.cluster_centers_
    vdict.append(np.zeros((128)))

    return vdict


In [ ]:
# Run the visual dict computation (saved the first time)
vdict = compute_or_load_vdict(dir_sc, dir_sift, inames, compute_sift_image, path_vdict, compute_visual_dict)

In [ ]:
# Study of the dict

# choose random images
indexes = np.random.randint(0, len(inames), 30)
sifts = [sifts_list_by_image[i] for i in indexes]
chosen_inames = [inames[i] for i in indexes]

regions, sifts = get_regions_and_sifts(dir_sc, chosen_inames, sifts) # Compute SIFT and regions from 30 random images
display_images(regions[np.random.choice(len(regions), 100)]) # Show 100 random regions

centers = list(range(20))
for center in centers:
  center_vect = vdict[center]
  dist = ((sifts - center_vect)**2).sum(axis=1)
  # get best 100
  top100 = dist.argsort()[:100]
  top100_regions = regions[top100]
  display_images(top100_regions) # Show 100 random regions
  

# Partie 3 : BoW

In [ ]:
def compute_feats(vdict, image_sifts):
    # flatten sifts
    sifts = image_sifts.reshape(-1, 128)  # (N, 128)
    feats = np.zeros(vdict.shape[0])

    for sift in sifts : 
      ind_min = np.argmin(distance_matrix(vdict, sift.reshape(1,128)))
      feats[ind_min] += 1

    feats = feats.reshape(1,-1)
    feats = normalize(feats, norm='l2')
    feats = feats[0]

    return feats

In [ ]:
# Visualize your BoW on an image
iname = inames[0]
ipath = os.path.join(dir_sc, iname)
im = read_grayscale(ipath)
sifts = compute_sift_image(im)
sifts = (sifts * 255).astype('uint8')
regions = compute_regions(im)
feats = compute_feats(vdict, sifts)

display_vdregions_image(im, vdict, sifts, feats, vdregions=None) # if you have vdregions, add it as input here

1) hx = (1, 2, 1) hy = (-1, 0, 1)
2) Permet parfois de faire moins d'étapes de calcul ?
3) Accorder plus d'importance aux gradients au centre du patch. Pour mieux différencier les types de sifts ? Image smoothing ?
4)  


